# TMA Capacitor Bank discharge vs. Acceleration profiles 
# [SITCOM-1223]

Laura Toribio San Cipriano
"Feb. 14, 2024"

https://jira.lsstcorp.org/browse/SITCOM-1223

**Description:**
Continue the analysis started in SITCOM-1146. Based on the input from a team review, focusing on aspects below. Another ticket opened to investigate current draw profiles.

- Relationship of min power supply voltage and distance of slew
- Explore when Az and El moved at same time, rather than separately
- Look at Azimuth acceleration vs. power supply voltage

Results and analysis to be added to SITCOMTN-110

### Setup

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits

from lsst_efd_client import EfdClient, resample
from lsst.summit.utils.tmaUtils import TMAEventMaker, TMAState

from astropy.time import Time, TimeDelta
from astropy.table import Table
from scipy.interpolate import UnivariateSpline

In [ ]:
client = EfdClient('usdf_efd')

In [ ]:
tma_power_topic = 'lsst.sal.MTMount.mainPowerSupply'
tma_el_topic = 'lsst.sal.MTMount.elevation'
tma_az_topic = 'lsst.sal.MTMount.azimuth'

### Get Data

Getting data corresponding to these:
 * [BLOCK-160](https://jira.lsstcorp.org/browse/BLOCK-160)
 * [BLOCK-164](https://jira.lsstcorp.org/browse/BLOCK-164)

In [ ]:
obs_days = [20231211, 20231212, 20231213, 20231214, 20231215, 20231218]

In [ ]:
def get_events(dayObs):
    """
    Identify slew events for a given dayObs
    """
    eventMaker = TMAEventMaker()
    events = eventMaker.getEvents(dayObs)
    
    # Get lists of slew and track events
    slews = [e for e in events if e.type == TMAState.SLEWING]
    tracks = [e for e in events if e.type == TMAState.TRACKING]
    print(f"{dayObs}: Found {len(slews)} slews and {len(tracks)} tracks")
    return slews

In [ ]:
def get_acc_jerk(time, velocity):
    """
    From velocity data, calculate acceleration and jerk
    """
    smoothing_factor=0.2
    kernel_size = 5
    kernel = np.ones(kernel_size) / kernel_size
    
    VelSpline = UnivariateSpline(time, velocity, s=0)
    smoothedVel = np.convolve(VelSpline(time), kernel, mode='same')
    VelSpline = UnivariateSpline(time, smoothedVel, s=smoothing_factor)

    AccSpline = VelSpline.derivative(n=1)
    smoothedAcc = np.convolve(AccSpline(time), kernel, mode='same')
    AccSpline = UnivariateSpline(time, smoothedAcc, s=smoothing_factor)
    print('AccSpline')
    print(len(AccSpline(time)))
    print('****')
    
    JerkSpline = AccSpline.derivative(n=1) 

    return VelSpline(time), AccSpline(time), JerkSpline(time)

#### Angular Distance Formula

WARNING: We have not been able to measure the distance in this way because the angles are too small and it gave us an error in the calculation.

The formula used to calculate the angular distance between two points on the celestial sphere is based on the Spherical Law of Cosines.

\[ \cos(\theta) = \sin(\text{elevation}_1) \sin(\text{elevation}_2) + \cos(\text{elevation}_1) \cos(\text{elevation}_2) \cos(\text{azimuth}_1 - \text{azimuth}_2) \]

Where:
- \( \theta \) is the angular distance between the two points on the celestial sphere.
- \( \text{elevation}_1 \) and \( \text{elevation}_2 \) are the elevations of the two points, respectively, measured in radians.
- \( \text{azimuth}_1 \) and \( \text{azimuth}_2 \) are the azimuths of the two points, respectively, measured in radians.

In [ ]:
# Function to calculate the angular distance between two points in the celestial sphere
def calculate_angular_distance(el_start, az_start, el_end, az_end):
    print(el_start)
    print(el_end)
    print(az_start)
    print(az_end)
    # Convert elevation and azimuth to radians
    el_start_rad = np.radians(el_start)
    az_start_rad = np.radians(az_start)
    el_end_rad = np.radians(el_end)
    a_end_rad = np.radians(az_end)
    # Calculate the angular distance using the cosine formula
    cos_theta = np.sin(el_start_rad) * np.sin(el_end_rad) + np.cos(el_start_rad) * np.cos(el_end_rad) * np.cos(az_start_rad - az_end_rad)
    print('3')
    print(cos_theta)
    
    # Calculate the angle (in radians) between the two points
    theta_rad = np.arccos(cos_theta)
    print('4')
    print(theta_rad)
    # Convert from radians to degrees and return the result
    return np.degrees(theta_rad)


In [ ]:
async def data_per_slew(i, slew, debug=False):
    """
    get TMA velocity data and power data for a given slew.

    Calculate maximum velocity, acceleration and jerk values and minimum power draw
    """
    data = [i, slew.dayObs, slew.duration, slew.begin, slew.end]
    power_df = await client.select_time_series(tma_power_topic, '*', slew.begin, slew.end)
    data.append(np.min(power_df.powerSupplyVoltage))
    print('power_df')
    print(power_df)

    az = await client.select_time_series(tma_az_topic, ['*'], slew.begin, slew.end)
    el = await client.select_time_series(tma_el_topic, ['*'], slew.begin, slew.end)

    AzBegin = az.actualPosition.iloc[0]
    AzEnd   = az.actualPosition.iloc[-1]
    ElBegin = el.actualPosition.iloc[0]
    ElEnd   = el.actualPosition.iloc[-1]

    #print('az')
    #print(az)
    AzVel, AzAcc, AzJerk = get_acc_jerk(az.timestamp, az.actualVelocity)
    ElVel, ElAcc, ElJerk = get_acc_jerk(el.timestamp, el.actualVelocity)

    #theta = calculate_angular_distance(ElBegin, AzBegin, ElEnd, AzEnd)
    
    for var in [AzVel, AzAcc, AzJerk, ElVel, ElAcc, ElJerk, AzBegin, AzEnd, ElBegin, ElEnd]: #, theta
        data.append(np.max(np.abs(var)))
    if debug:
        return az, el, power_df, data
    else:
        return data

In [ ]:
Data = []
failed = []
data_names = ['slew_id','dayobs','duration','begin','end','min_power','az_max_vel','az_max_acc','az_max_jerk','el_max_vel','el_max_acc','el_max_jerk', 'az_pos_begin', 'az_pos_end', 'el_pos_begin', 'el_pos_end'] #, 'dis_slew'
for dayObs in obs_days:
    try:
        slews = get_events(dayObs)
    except:
        print(dayObs)
    for i, slew in enumerate(slews):
        try:
            d = await data_per_slew(i, slew)
            Data.append(d)
        except Exception as e:
            failed.append(i)
            #print(i, e)
df = pd.DataFrame(np.vstack(Data),columns=data_names)
len(df)

In [ ]:
#Remove extreme outliers
df = df[(df.el_max_jerk < 100)&(df.duration<3000)]

### Analysis

Relationship of min power supply voltage and distance of slew

In [ ]:
def plot_psv_vs_dist(df_, title_date,name):
    fig, ax = plt.subplots(1, 1, figsize=(6, 4))

    ax.plot((df_.az_pos_begin - df_.az_pos_end), df_.min_power,'x', color='C0', label='azPos')
    ax.plot((df_.el_pos_begin - df_.el_pos_end), df_.min_power,'x', color='C1', label='elPos')
    #ax1.axhline(575, ls='--', c='k',label='voltage drop limit')
    ax.legend()
    #ax1.set_xlim(-1,12)
    ax.set_xlabel('distance of slew')
    ax.set_ylabel('min supplyPowerVoltage (V)')

    plt.savefig(f'{name}.png')

In [ ]:
plot_psv_vs_dist(df, 'BLOCK-160/164','tma_perf_dec_2023')

Look at Azimuth acceleration vs. power supply voltage